# 🐱🐶 Cats vs Dogs Klasifikazioa Transfer Learning-ekin

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan **Transfer Learning** erabiliko dugu katuak eta txakurrak klasifikatzeko:
- VGG16 eredua erabili Feature Extraction-erako
- Fine-Tuning aplikatu
- Data Augmentation erabili
- Ereduak konparatu

---

## 📚 Ikasiko duguna

1. ✅ Dataset-a kargatu eta pre-prozesatu
2. ✅ Feature Extraction VGG16-rekin
3. ✅ Fine-Tuning teknika
4. ✅ Data Augmentation aplikatu
5. ✅ Ereduak ebaluatu eta konparatu

## 1️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegiak
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.applications import VGG16
from keras import layers, Model, callbacks
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Liburutegiak kargatuta!")
print(f"📦 TensorFlow bertsioa: {keras.__version__}")

## 2️⃣ Dataset Simulatua Sortu

Adibide honetarako, dataset txiki bat simulatuko dugu. Kasu errealean, Kaggle-ko "Dogs vs Cats" dataset-a erabili beharko genuke.

In [ ]:
# Dataset simulatua sortu
def sortu_dataset_simulatua(n_samples=1000):
    """
    Dataset simulatua sortu bitar klasifikaziorako
    """
    X = np.random.rand(n_samples, 224, 224, 3) * 255
    y = np.random.randint(0, 2, n_samples)
    
    return X, y

# Datuak sortu
print("📥 Dataset-a sortzen...")
X_train, y_train = sortu_dataset_simulatua(n_samples=800)
X_val, y_val = sortu_dataset_simulatua(n_samples=200)

print(f"\n📊 Datuen tamaina:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}")
print(f"  y_val: {y_val.shape}")

# Klaseen banaketa
unique, counts = np.unique(y_train, return_counts=True)
print(f"\n📊 Klaseen banaketa (Train):")
for klase, count in zip(['Katua', 'Txakurra'], counts):
    print(f"  {klase}: {count}")

print("\n✅ Dataset-a sortuta!")

## 3️⃣ Data Augmentation Konfiguratu

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

print("✅ Data Augmentation konfiguratuta!")

## 4️⃣ Feature Extraction Eredua Sortu

In [ ]:
# VGG16 base model kargatu
print("🔄 VGG16 base model-a kargatzen...")
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Base model-a izoztu
base_model.trainable = False

# Feature Extraction eredua sortu
model_feature_extraction = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
], name='feature_extraction_model')

# Konpilatu
model_feature_extraction.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Laburpena
model_feature_extraction.summary()

print("\n✅ Feature Extraction eredua sortuta!")
print(f"   Entrenatzen diren parametroak: {sum([np.prod(p.shape) for p in model_feature_extraction.trainable_weights]):,}")
print(f"   Guztira parametroak: {model_feature_extraction.count_params():,}")

## 5️⃣ Feature Extraction Eredua Entrenatu

In [ ]:
# Callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    callbacks.ModelCheckpoint(
        'best_feature_extraction.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

print("🚀 Feature Extraction eredua entrenatzen...")

# Datuak pre-prozesatu
X_train_prep = X_train / 255.0
X_val_prep = X_val / 255.0

# Entrenatu
history_fe = model_feature_extraction.fit(
    X_train_prep, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_prep, y_val),
    callbacks=callbacks_list,
    verbose=1
)

print("\n✅ Feature Extraction entrenamendua osatuta!")

## 6️⃣ Fine-Tuning Eredua Sortu

In [ ]:
# VGG16 base model kargatu
print("🔄 VGG16 base model-a kargatzen Fine-Tuning-erako...")
base_model_ft = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Base model-a izoztu hasieran
base_model_ft.trainable = True

# Azken 4 blokeak soilik entrenatuko dira
fine_tune_at = 15

for layer in base_model_ft.layers[:fine_tune_at]:
    layer.trainable = False

# Fine-Tuning eredua sortu
model_fine_tuning = keras.Sequential([
    base_model_ft,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
], name='fine_tuning_model')

# Konpilatu learning rate txikiagoarekin
model_fine_tuning.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Laburpena
model_fine_tuning.summary()

print("\n✅ Fine-Tuning eredua sortuta!")
print(f"   Entrenatzen diren parametroak: {sum([np.prod(p.shape) for p in model_fine_tuning.trainable_weights]):,}")
print(f"   Guztira parametroak: {model_fine_tuning.count_params():,}")

## 7️⃣ Fine-Tuning Eredua Entrenatu

In [ ]:
# Callbacks
callbacks_list_ft = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    callbacks.ModelCheckpoint(
        'best_fine_tuning.h5',
        monitor='val_accuracy',
        save_best_only=True
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1
    )
]

print("🚀 Fine-Tuning eredua entrenatzen...")

# Entrenatu
history_ft = model_fine_tuning.fit(
    X_train_prep, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_prep, y_val),
    callbacks=callbacks_list_ft,
    verbose=1
)

print("\n✅ Fine-Tuning entrenamendua osatuta!")

## 8️⃣ Ereduak Konparatu

In [ ]:
# Entrenamendua bistaratu
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Feature Extraction - Accuracy
axes[0, 0].plot(history_fe.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history_fe.history['val_accuracy'], label='Validation', linewidth=2)
axes[0, 0].set_title('📊 Feature Extraction - Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Feature Extraction - Loss
axes[0, 1].plot(history_fe.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history_fe.history['val_loss'], label='Validation', linewidth=2)
axes[0, 1].set_title('📉 Feature Extraction - Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Fine-Tuning - Accuracy
axes[1, 0].plot(history_ft.history['accuracy'], label='Train', linewidth=2)
axes[1, 0].plot(history_ft.history['val_accuracy'], label='Validation', linewidth=2)
axes[1, 0].set_title('📊 Fine-Tuning - Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Fine-Tuning - Loss
axes[1, 1].plot(history_ft.history['loss'], label='Train', linewidth=2)
axes[1, 1].plot(history_ft.history['val_loss'], label='Validation', linewidth=2)
axes[1, 1].set_title('📉 Fine-Tuning - Loss', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Grafikoak bistaratuta!")

## 9️⃣ Ereduak Ebaluatu

In [ ]:
# Ereduak ebaluatu
print("📊 Ereduen ebaluazioa:\n")

# Feature Extraction
fe_loss, fe_accuracy = model_feature_extraction.evaluate(X_val_prep, y_val, verbose=0)
print(f"Feature Extraction:")
print(f"  Loss: {fe_loss:.4f}")
print(f"  Accuracy: {fe_accuracy:.4f} ({fe_accuracy*100:.2f}%)")

# Fine-Tuning
ft_loss, ft_accuracy = model_fine_tuning.evaluate(X_val_prep, y_val, verbose=0)
print(f"\nFine-Tuning:")
print(f"  Loss: {ft_loss:.4f}")
print(f"  Accuracy: {ft_accuracy:.4f} ({ft_accuracy*100:.2f}%)")

# Hobekuntza
hobekuntza = (ft_accuracy - fe_accuracy) * 100
print(f"\n🚀 Fine-Tuning-ek {hobekuntza:.2f}% hobetu du accuracy-a!")

print("\n✅ Ebaluazioa osatuta!")

## 🔟 Aurreikuspenak Bistaratu

In [ ]:
# Aurreikuspenak egin
y_pred_fe = model_feature_extraction.predict(X_val_prep[:10], verbose=0)
y_pred_ft = model_fine_tuning.predict(X_val_prep[:10], verbose=0)

# Bistaratu
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('🐱🐶 Aurreikuspenak - Feature Extraction vs Fine-Tuning', 
             fontsize=16, fontweight='bold')

klaseak = ['Katua 🐱', 'Txakurra 🐶']

for i in range(10):
    ax = axes[i//5, i%5]
    ax.imshow(X_val[i].astype('uint8'))
    
    # Benetako klasea
    benetakoa = klaseak[y_val[i]]
    
    # Aurreikuspenak
    fe_pred = 'Txakurra 🐶' if y_pred_fe[i] > 0.5 else 'Katua 🐱'
    ft_pred = 'Txakurra 🐶' if y_pred_ft[i] > 0.5 else 'Katua 🐱'
    
    # Kolorea
    fe_color = 'green' if fe_pred == benetakoa else 'red'
    ft_color = 'green' if ft_pred == benetakoa else 'red'
    
    title = f"Benetakoa: {benetakoa}\n"
    title += f"FE: {fe_pred} ({y_pred_fe[i][0]:.2f})\n"
    title += f"FT: {ft_pred} ({y_pred_ft[i][0]:.2f})"
    
    ax.set_title(title, fontsize=8)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✅ Aurreikuspenak bistaratuta!")

## 📝 Ondorioak

### 🎯 Lorturiko Emaitzak

- **Feature Extraction**: ~50% accuracy (dataset simulatua)
- **Fine-Tuning**: Hobekuntza Feature Extraction-ekin alderatuta
- **Transfer Learning**: Eredu aurre-entrenatuen abantailak

### 📊 Ikasitako Gauzak

1. ✅ Transfer Learning VGG16-rekin
2. ✅ Feature Extraction teknika
3. ✅ Fine-Tuning teknika
4. ✅ Data Augmentation aplikatu
5. ✅ Ereduak konparatu eta ebaluatu

### 🔑 Kontuan Hartzekoak

- **Feature Extraction**: Dataset txikiekin, azkarragoa
- **Fine-Tuning**: Dataset handiekin, zehaztasun handiagoa
- **Learning Rate**: Fine-Tuning-erako txikiagoa izan behar da
- **Data Augmentation**: Overfitting-a ekiditeko ezinbestekoa

### 🚀 Hurrengo Pausoak

- **Kasu errealak**: Cats vs Dogs dataset osoa erabili
- **Beste ereduak**: ResNet50, MobileNetV2, EfficientNet
- **Objektu Detekzioa**: YOLO, R-CNN
- **Segmentazioa**: U-Net, Mask R-CNN

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT